In [ ]:
!pip -q install streamlit "transformers>=4.43" "peft>=0.12" "accelerate>=0.33" "bitsandbytes>=0.43" "pypdf>=4.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip -q install cloudflared  # (Option B: cloudflared)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Get the latest linux-amd64 binary and put it on PATH
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
!cloudflared --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 39.2M    0 16375    0     0  22937      0  0:29:55 --:--:--  0:29:55 22937
curl: (23) Failure writing output to destination
cloudflared version 2025.8.0 (built 2025-08-08-1545 UTC)


In [ ]:
# ✅ Kill any previous Streamlit / cloudflared processes
!pkill -f "streamlit run /content/sample_data/streamlit_colab_inference_mapping_ui.py" || true
!pkill -f cloudflared || true

# ✅ Ensure a recent, compatible cloudflared
!pip -q install -U cloudflared

# ✅ Start Streamlit in the background on port 8501
!nohup streamlit run /content/sample_data/streamlit_colab_inference_mapping_ui.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &

# ✅ Start cloudflared in the background and log to a file
!nohup cloudflared tunnel --no-autoupdate --loglevel info --url http://localhost:8501 > cloudflared.log 2>&1 &

import re, time, os, itertools

log_path = "cloudflared.log"
public_url = None
print("⏳ Waiting for the public URL from cloudflared…")

# Poll the logfile for up to 60s
for _ in itertools.repeat(None, 60):
    if os.path.exists(log_path):
        with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
            m = re.search(r"(https://[a-z0-9-]+\.trycloudflare\.com)", text)
            if m:
                public_url = m.group(1)
                break
    time.sleep(1)

if public_url:
    print("\n✅ Public URL (open in your browser):", public_url, "\n")
else:
    print("\n❌ Could not detect the trycloudflare URL after 60s.")
    print("🪵 Last 120 lines of cloudflared.log for debugging:\n")
    !tail -n 120 cloudflared.log || true
    print("\n💡 Tips:")
    print(" - Re-run this cell once; sometimes the first start is slow.")
    print(" - If your region is flaky, try ngrok (next cell) as a fallback.")


^C
^C
⏳ Waiting for the public URL from cloudflared…

✅ Public URL (open in your browser): https://ascii-downloaded-resources-says.trycloudflare.com 



In [ ]:
#dual adapter with tool chain execution
# ✅ Kill any previous Streamlit / cloudflared processes
!pkill -f "streamlit run /content/sample_data/streamlit_colab_dual_adapter_ui.py" || true
!pkill -f cloudflared || true

# ✅ Ensure a recent, compatible cloudflared
!pip -q install -U cloudflared

# ✅ Start Streamlit in the background on port 8501
!nohup streamlit run /content/sample_data/streamlit_colab_dual_adapter_ui.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &

# ✅ Start cloudflared in the background and log to a file
!nohup cloudflared tunnel --no-autoupdate --loglevel info --url http://localhost:8501 > cloudflared.log 2>&1 &

import re, time, os, itertools

log_path = "cloudflared.log"
public_url = None
print("⏳ Waiting for the public URL from cloudflared…")

# Poll the logfile for up to 60s
for _ in itertools.repeat(None, 60):
    if os.path.exists(log_path):
        with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
            m = re.search(r"(https://[a-z0-9-]+\.trycloudflare\.com)", text)
            if m:
                public_url = m.group(1)
                break
    time.sleep(1)

if public_url:
    print("\n✅ Public URL (open in your browser):", public_url, "\n")
else:
    print("\n❌ Could not detect the trycloudflare URL after 60s.")
    print("🪵 Last 120 lines of cloudflared.log for debugging:\n")
    !tail -n 120 cloudflared.log || true
    print("\n💡 Tips:")
    print(" - Re-run this cell once; sometimes the first start is slow.")
    print(" - If your region is flaky, try ngrok (next cell) as a fallback.")


^C
^C
⏳ Waiting for the public URL from cloudflared…

✅ Public URL (open in your browser): https://force-humans-manner-dose.trycloudflare.com 

